In [74]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [75]:
var offset = "earliest"
//val test: Int = 20

if (offset != "earliest") {
    offset = s"""{"$topic_name":{"0":$offset}}"""
}

offset = earliest


earliest

In [76]:
val topic_name = "lab04_input_data"

val output_dir_prefix = "/user/andrey.blednykh2/visits"

topic_name = lab04_input_data
test = 20
output_dir_prefix = /user/andrey.blednykh2/visits


/user/andrey.blednykh2/visits

In [78]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "spark-master-1:6667",
        "subscribe" -> topic_name,
        "startingOffsets" -> offset,
        "endingOffsets" -> latest
    )

Name: Compile Error
Message: <console>:63: error: not found: value latest
               "endingOffsets" -> latest
                                  ^

StackTrace: 

In [79]:
val df_kafka = spark
    .read
    .format("kafka")
    .options(kafkaParams)
    .load

df_kafka = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

In [81]:
val parsed_df_kafka = df_kafka
    .select('key, 'value.cast("string"), 'topic, 'partition, 'offset, 'timestamp, 'timestampType)

parsed_df_kafka = [key: binary, value: string ... 5 more fields]


[key: binary, value: string ... 5 more fields]

In [82]:
val value_df_kafka = parsed_df_kafka.select('value.cast("string"))

value_df_kafka = [value: string]


[value: string]

In [83]:
val schema = StructType(Seq(
    StructField("event_type", StringType, true),    
    StructField("category", StringType, true),
    StructField("item_id", StringType, true),
    StructField("item_price", IntegerType, true),
    StructField("uid", StringType, true),
    //StructField("timestamp", StringType, true)
    //StructField("timestamp", TimestampType, true)
    StructField("timestamp", LongType, true)
))

schema = StructType(StructField(event_type,StringType,true), StructField(category,StringType,true), StructField(item_id,StringType,true), StructField(item_price,IntegerType,true), StructField(uid,StringType,true), StructField(timestamp,LongType,true))


StructType(StructField(event_type,StringType,true), StructField(category,StringType,true), StructField(item_id,StringType,true), StructField(item_price,IntegerType,true), StructField(uid,StringType,true), StructField(timestamp,LongType,true))

In [84]:
val json_df_kafka = value_df_kafka
    .withColumn("value", from_json($"value", schema))
    .select($"value.*")

json_df_kafka = [event_type: string, category: string ... 4 more fields]


[event_type: string, category: string ... 4 more fields]

In [ ]:
json_df_kafka.show(1,200,true)

In [85]:
val k_date = json_df_kafka
    .withColumn("date", 
                trim(date_format(to_timestamp((col("timestamp")/1000).cast("long")),"yyyyMMdd")))
    .withColumn("p_date", 
                trim(date_format(to_timestamp((col("timestamp")/1000).cast("long")),"yyyyMMdd")))
    .cache()

k_date = [event_type: string, category: string ... 6 more fields]


[event_type: string, category: string ... 6 more fields]

In [86]:
val k_view_date = k_date
    .select("category","event_type","item_id","item_price","timestamp","uid","date","p_date")
    .filter("event_type = 'view'")
    //.na.drop(Seq("uid"))
    //.filter($"uid" =!= "unknown")

k_view_date = [category: string, event_type: string ... 6 more fields]


[category: string, event_type: string ... 6 more fields]

In [ ]:
k_view_date.count

In [ ]:
k_view_date.show(200, 200, true)

In [87]:
val k_buy_date = k_date
    .select("category","event_type","item_id","item_price","timestamp","uid","date","p_date")
    .filter("event_type = 'buy'")
    //.na.drop(Seq("uid"))
    //.filter($"uid" =!= "unknown")

k_buy_date = [category: string, event_type: string ... 6 more fields]


[category: string, event_type: string ... 6 more fields]

In [ ]:
k_buy_date.count

In [ ]:
k_buy_date.show(10, 200, true)

In [88]:
//import spark.implicits._
k_view_date
    //.repartition($"p_date")
    .write
    .mode("overwrite")
    .partitionBy("p_date")
    .json(s"$output_dir_prefix/view/")

In [89]:
//import spark.implicits._
k_buy_date
    //.repartition($"p_date")
    .write
    .mode("overwrite")
    .partitionBy("p_date")
    .json(s"$output_dir_prefix/buy/")